In [ ]:
import pandas as pd
import pandas as pd
import geopandas as gpd
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
#from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [ ]:
data = pd.read_csv("tanzania2023.csv")
data['harvest'] = pd.to_datetime(data['harvest'], format='%d/%m/%Y')
#data['harvest_month'] = df_date.dt.month_name()
data['harvest_month'] = data['harvest'].dt.strftime("%B")
data['harvest_year'] = data['harvest'].dt.strftime("%Y")
#data.to_csv("updated_tanzania23.csv", index=False)

print(data.head(10))

In [35]:
months = ['May']
df22 = data.loc[data['harvest_year'] != 2023]
#df22 = data[data['harvest_month'].isin(months)]
#df22 = df.groupby('cname').mean()
print(df22.head(10))
#max(df22['wsgd'])

  cname    harvest    planting  wsgd  gwad harvest_month harvest_year
0  Bahi 2022-03-06  04/11/2021   0.0  2897         March         2022
1  Bahi 2022-02-23  19/10/2021   0.0  3978      February         2022
2  Bahi 2022-02-23  19/10/2021   0.0  3975      February         2022
3  Bahi 2022-03-06  04/11/2021   0.0  2899         March         2022
4  Bahi 2022-02-18  13/10/2021   0.0  4251      February         2022
5  Bahi 2022-02-11  08/10/2021   0.0  4059      February         2022
6  Bahi 2022-03-06  04/11/2021   0.0  2903         March         2022
7  Bahi 2022-02-13  10/10/2021   0.0  4158      February         2022
8  Bahi 2022-02-13  10/10/2021   0.0  4158      February         2022
9  Bahi 2022-02-23  19/10/2021   0.0  4017      February         2022


In [46]:
grouped = data.groupby(["cname", "harvest_year", "harvest_month"])["wsgd"].mean()
print(grouped.head(20))


cname     harvest_year  harvest_month
Bahi      2022          February         0.0000
                        July             0.0000
                        March            0.0000
          2023          February         0.0000
                        June             0.0401
                        March            0.0000
Buhigwe   2022          February         0.0000
                        January          0.0000
                        July             0.5138
          2023          February         0.0000
                        January          0.0000
                        June             0.0989
Chamwino  2022          February         0.0000
                        July             0.0000
                        March            0.0000
          2023          February         0.0000
                        June             0.1753
                        March            0.0000
Chemba    2022          February         0.0000
                        July             0.0000
Na

In [ ]:
shapefile = gpd.read_file("unimodal.shp")
country_shapefile = gpd.read_file('tanzania.shp')
#shapefile["NewDist20"] = shapefile.index
#print(shapefile)

In [ ]:
yield_data = pd.read_csv("districts_points2.csv")
#print(yield_data)

In [ ]:
merged_data = pd.merge(shapefile, yield_data, on="NAME_2")
#print(merged_data)

In [ ]:
geo_source = GeoJSONDataSource(geojson=merged_data.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [ ]:
color_mapper = LinearColorMapper(palette=Viridis256, low=min(merged_data['gwad']), high=max(merged_data['gwad']))

In [ ]:
p1 = figure(title='Yield Data by District',
           plot_width=650, plot_height=600)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source,
          fill_color={'field': 'gwad', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@NAME_2'), ('gwad', '@gwad')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [ ]:
color_mapper1 = LinearColorMapper(palette=Viridis256, low=min(merged_data['wsgd']), high=max(merged_data['wsgd']))

p2 = figure(title='Yield Data by District',
           plot_width=650, plot_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_source,
          fill_color={'field': 'wsgd', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@NAME_2'), ('wsgd', '@wsgd')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [ ]:
from bokeh.layouts import column
layout = column(p1, p2)
#show(layout)

In [ ]:
from bokeh.io import output_file, show
output_file("plot.html")
show(layout)

In [ ]:
#from bokeh.io import save

# specify the output file name and location
#save(p, filename='yield_map.html', title='Yield Data Map')